Introduction
===
This notebook filters and sets up the input traces for the
Matlab model building script.

Setup
---
Set the working directory below. The working directory should 
include the signal (power values) traces and trigger traces. 
This is generated by running the output of `genasm.py <state>` 
on a device. These files should be in their respective directories
according to what state changes applied to generate them (i.e.
files that are related to `ldr` is in `ldr-state` in the working
directory). If these files need to be averaged for noise reduction
please put them in `<working-dir>/<xxx-state>/run_N`, where `N` is
an starts with `0`. Use `BinTraces.merge` to load the them, otherwise
concatenate all the runs to a numpy array.

In [1]:
import mmap
import struct
import os
import numpy
import importlib
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# include analyze_traces repo as we 
# depend on some modules from it
sys.path.append('/home/samiko/Desktop/rosita/analyze_traces')
import BinTraces
import Pool

# current working directory, the traces are loaded from here
os.chdir("/home/samiko/Desktop/rosita/cwmodel/") 
importlib.reload(BinTraces)

<module 'BinTraces' from '/home/samiko/Desktop/rosita/analyze_traces/BinTraces.py'>

In [7]:
# Notes:
# A array contains the signal data
# B array contains the trigger data where the signals start
# state_dir is the current state used on genasm.py (i.e. genasm.py ldr-state)
#A = BinTraces.BinTraces("capture_A.xrc")
#B = BinTraces.BinTraces("capture_B.xrc")

# Trace sources for the state under test
# pick one at a time
state_dir = "ldr-state"
#state_dir = "eors-state"
#state_dir = "str-state"
#state_dir = "movs-state"
#state_dir = "no-state"

#A = BinTraces.merge(state_dir, "capture_A.xrc")
#B = BinTraces.merge(state_dir, "capture_B.xrc")

SAMPLES_PER_CYCLE = 9.76
# number of instruction state classes (i.e. ldr,str,eors,movs,none)
nI = 5
nIC = 5*5*5
# number of samples in Point of Interest: default is 200 sample from the start
# of an trigger
nPOISamples = 200 
# number of traces to process 
N = 1000

# following is hardcoded values for the begining of 
# each instruction triplet, according to the output
# of `genasm.py`
if state_dir.startswith('ldr-state'):
    # (11+5)(trigger and pop) +(1+1+2+1)(ldr values)+(5)(movs r7,r7 x5)
    CYCLES_FROM_TRIGGER=11+5+(1+1+1+2+1+1+1+2+1)+(5) 
elif state_dir.startswith('str-state'):
    CYCLES_FROM_TRIGGER=11+5+(1+2+1+2+1+1+1+2+1)+(5) 
elif state_dir.startswith('eors-state'):
    CYCLES_FROM_TRIGGER=11+5+(1+2+1+2+2+1+1+1+1)+(5)
elif state_dir.startswith('movs-state'):
    CYCLES_FROM_TRIGGER=11+5+(1+2+1+2+2+1+1+1+1)+(5)
elif state_dir.startswith('no-state'):
    CYCLES_FROM_TRIGGER=11+5+(1+2+1+2+2+1+1+1+1)+(5)
else:
    raise BaseException("invalid state")
    
print(CYCLES_FROM_TRIGGER)

33


In [3]:
#raise

def trace_thres(work):
    ctx, start, end = work
    POI_S = []
    for idx in range(start, end):
        xb=B.read_trace(idx)
        t=1500
        #V1 = (xb[:-1] < t) & (xb[1:] > t)
        V2 = (xb[:-1] > t) & (xb[1:] < t)

        #C = np.flatnonzero(V1 | V2) +1
        C = np.flatnonzero(V2) + 2 # falling edge

        assert np.shape(C)[0] == 125, np.shape(C)[0]

        # after trigger + pop instrs
        # offset to first instruction after falling edge
        POI_S.append( C + int(round(CYCLES_FROM_TRIGGER*SAMPLES_PER_CYCLE)) ) 
    return POI_S

p = Pool.Mapper(8, 1000, trace_thres)
S = p.merge([], lambda acc,x: acc.extend(x))
np.save("%s/POI_S.npy" % (state_dir), np.asarray(S), allow_pickle=False);

In [4]:
POI_S = np.load("%s/POI_S.npy"% (state_dir))
print("loaded from disk", state_dir, POI_S.shape)

loaded from disk ldr-state (1000, 125)


In [6]:
# Since there can be discrepancies in averaging
# the whole signal traces without aligning and splitting
# by the triggers, we instead average on the split 
# signals
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

AVG = np.zeros((N, nIC, nPOISamples), dtype=np.float)
xN = np.shape(POI_S)[0] // N
print("averaging %d times" % (xN))
vv =[]
divk = 1.0/xN
for k in range(0, N): # avg trace
    POI_wind_total = np.zeros((nIC, nPOISamples), dtype=np.float)
    temp = np.zeros((nPOISamples), dtype=np.float)
    for j in range(k, np.shape(POI_S)[0], N): # trace
        for i in range(0, nIC):
            #sliding_wind(temp, A.read(j, POI_S[j][i], nPOISamples), 8)
            POI_wind = A.read(j, POI_S[j][i], nPOISamples)
            POI_wind_total[i,:] += POI_wind
        AVG[k,:] = POI_wind_total * divk
    if k % 128 == 0:
        print(k,"/",N)


averaging 1 times
0 / 1000
128 / 1000
256 / 1000
384 / 1000
512 / 1000
640 / 1000
768 / 1000
896 / 1000


In [7]:
print("saving intermediates to %s"%state_dir)
np.save("%s/AVG.npy"% (state_dir), AVG)
#np.save("%s/POI_S.npy"% (state_dir), POI_S);

saving intermediates to ldr-state


In [8]:
# Calculate the start and end of intructions
# in samples and then compress the sample ranges
# in to single values (COMP will hold the 
# compressed values)
K=SAMPLES_PER_CYCLE
COMP = np.zeros((N, nIC, 3), dtype=np.float)
#STATE = np.zeros((N,4), dtype=np.float)
#GGG = np.zeros((N,200), dtype=np.float)
Nx = np.shape(POI_S)[0] / N


for i in range(0, N):
    COMPx = np.zeros((nIC, 3), dtype=np.float)
    # 125 permutations + 4 state transisions
    # tr = A.read_trace(i)
    for j in range(0, nIC): 
        insts = INSTS[j]
        avg = AVG[i, j]
        cycles_s = [0, insts[0]*K, (insts[0]+insts[1])*K]
        cycles_e = [insts[0]*K, (insts[0]+insts[1])*K, (insts[0]+insts[1]+insts[2])*K]
        
        comp = []
        for rr in range(0, 3):
            tr = avg[int(round(cycles_s[rr])):int(round(cycles_e[rr]))]
            comp.append(np.max(tr))
            
        COMPx[j] = np.asarray(comp)
    COMP[i] = COMPx

In [7]:
import numpy as np
import os, sys
trace_array = np.load("/home/samiko/Desktop/rosita/cwmodel/ldr-state/ldr_state.npy")
trace_array.shape

(125, 10, 2000)

In [9]:

assert np.shape(COMP)[0] == N 
for i in range(0, N):
    assert np.shape(COMP[i])[0] == nIC
inputs = [x.rstrip('\n') for x in open("%s/run_0/prf_input.txt"%(state_dir)).readlines()]
inputs = inputs[:N]


In [10]:
v = lambda x: ((int(x[0:8],16),int(x[8:16],16)),
               (int(x[16:24],16),int(x[24:32],16)),
               (int(x[32:40],16),int(x[40:48],16)),
               (int(x[48:56],16),int(x[56:64],16)))
npV = lambda x: [int(x[0:8],16),int(x[8:16],16),int(x[16:24],16),int(x[24:32],16),int(x[32:40],16),int(x[40:48],16),int(x[48:56],16),int(x[56:64],16)]
INPUTS = list(map(v, inputs))
npINPUTS = np.asarray(list(map(npV, inputs)))

inputbytes = list(map(bytes.fromhex, inputs))
splitbytes = lambda x: struct.unpack('B'*32, x)

inputints = np.asarray(list(map(splitbytes, inputbytes)),dtype=np.uint32)
print(inputints)

[[195   5 113 ...  22 125  78]
 [165 155 169 ...  46  23 128]
 [200 239 139 ...  84 247 240]
 ...
 [222 105 190 ... 162  71 173]
 [159 202  82 ... 248 189 212]
 [246 230 128 ...  36  77 192]]


In [3]:
import random
def _m(k):
    return k & 0xff
def _hw(k):
    k = k & 0xffffffff
    return bin(k).count('1')
def _hd(k,l):
    return _hw(k ^ l)
def prng(x):
    x = (1103515245*x + 12345) & 0xffffffff;
    return x
    #return (241*x + 17) & 0xff
def incr(seeds):
    seeds = seeds & 0xffffffff
    #return (_m(_m(cc >> 24) +1)<<24) | (_m(_m(cc >> 16) +1)<<16) | (_m(_m(cc >> 8) +1)<<8)  | (_m(_m(cc) +1)) 
    seeds[0], b8_0 = func256(seeds[0])
    seeds[1], b8_1 = func256(seeds[1])
    seeds[2], b8_2 = func256(seeds[2])
    seeds[3], b8_3 = func256(seeds[3])
    return (b8_0<<24) | (b8_1<<16) | (b8_2<<8) | (b8_3) 

print(prng(inputints)>>24)

NameError: name 'inputints' is not defined

In [12]:
np.save('%s-comp.npy'%(state_dir),COMP,allow_pickle=False)
np.save('%s-input.npy'%(state_dir),INPUTS,allow_pickle=False)
np.save('%s-opcodes.npy'%(state_dir),OPCODES,allow_pickle=False)

In [13]:
assert len(INSTS) == nIC, len(INSTS)
assert len(OPCODES) == nIC, len(OPCODES)
assert len(COMP) == N, len(COMP)

In [31]:
from tqdm.notebook import tnrange
import numpy as np
import os, sys
import glob

states = ['ldr-state', 'str-state', 'eors-state', 'movs-state', 'no-state']

N = 5
for i in tnrange(N, desc='Merging traces'):
    basepath = "/home/samiko/Desktop/rosita/cwmodel/{}/".format(states[i])
    npypath = "/home/samiko/Desktop/rosita/cwmodel/{}/traces".format(states[i])

    os.chdir(npypath)
    npfiles = glob.glob("*.npy")
    npfiles.sort()

    trace_array = []
    for npfile in npfiles:
        trace_array.append(np.load(os.path.join(npypath, npfile)))

    trace_array = np.array(trace_array)  
    print(trace_array.shape)

    os.chdir(basepath)
    with open("{}.npy".format(states[i].replace('-','_')), 'wb') as f:
        np.save(f, trace_array)

Merging traces:   0%|          | 0/5 [00:00<?, ?it/s]

(125, 10, 2000)
(125, 10, 2000)
(125, 10, 2000)
(125, 10, 2000)
(125, 10, 2000)


In [27]:
# from ELMO model
OPCODES_S = ['eors','lsls','str','ldr','movs'] 
# from state leakage matrix 
STATE_S = ['eors','str','ldr','movs'] 
insts_len = [1,1,2,2,1]
INSTS = []

OPCODES = []
idx = 0
eors_idxes = []
N_TEST = 10
N_INST = 5*5*5

def insts_part():
    return  range(0, len(insts_len))
for idx1 in insts_part():
    for idx2 in insts_part():
        for idx3 in insts_part():
            OPCODES += [(idx1+1,idx2+1,idx3+1)]
            #if (OPCODES_S[idx1] == "ldr"):
            #    print(idx, OPCODES_S[idx1], OPCODES_S[idx2], OPCODES_S[idx3])
            INSTS += [(insts_len[idx1],insts_len[idx2],insts_len[idx3])]
            idx +=1

nOPCODES = np.zeros((N_R,3))
for i in range(0, N_INST):
    for j in range(0, N_TEST):
        nOPCODES[i*N_TEST + j][0] = OPCODES[i][0]
        nOPCODES[i*N_TEST + j][1] = OPCODES[i][1]
        nOPCODES[i*N_TEST + j][2] = OPCODES[i][2]

print(nOPCODES)
print(nOPCODES.shape)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [5. 5. 5.]
 [5. 5. 5.]
 [5. 5. 5.]]
(1250, 3)


In [39]:
from tqdm.notebook import tnrange
import numpy as np
import os, sys

N_TEST= 10
N_INST= 5*5*5

N_R = N_INST*N_TEST

trace_array = np.load("/home/samiko/Desktop/rosita/cwmodel/no-state/no_state.npy")

nTRACES = np.zeros((N_R,1), dtype=np.double)
w = 0
for i in range(0, N_INST):
    for j in range(0, N_TEST):
        nTRACES[w] = -np.mean(trace_array[i][j][150:170])
        w += 1

print(nTRACES)
print(nTRACES.shape)

[[0.04453125]
 [0.04277344]
 [0.0421875 ]
 ...
 [0.04453125]
 [0.04179687]
 [0.04140625]]
(1250, 1)


In [40]:
from random import randint

N_TEST= 10
N_INST= 5*5*5

N_R = N_INST*N_TEST

nINPUT1 = np.zeros((N_R,2), dtype=np.double)
nINPUT2 = np.zeros((N_R,2), dtype=np.double)
nINPUT3 = np.zeros((N_R,2), dtype=np.double)
nSTATEINP = np.zeros((N_R,2), dtype=np.double)

for i in range(0,N_R):
    nINPUT1[i][0] = randint(10000000, 999999999)
    nINPUT1[i][1] = randint(10000000, 999999999)
    nINPUT2[i][0] = randint(10000000, 999999999)
    nINPUT2[i][1] = randint(10000000, 999999999)
    nINPUT3[i][0] = randint(10000000, 999999999)
    nINPUT3[i][1] = randint(10000000, 999999999)
    nSTATEINP[i][0] = randint(10000000, 999999999)

In [41]:
import scipy.io

OPCODES_S = ['eors','lsls','str','ldr','movs'] 

_c = np.zeros((1,3), dtype=np.object)
_c[0][0] = nINPUT1
_c[0][1] = nINPUT2
_c[0][2] = nINPUT3

_s = np.zeros((1,5), dtype=np.object)
_s[0][0] = OPCODES_S[0]
_s[0][1] = OPCODES_S[1]
_s[0][2] = OPCODES_S[2]
_s[0][3] = OPCODES_S[3]
_s[0][4] = OPCODES_S[4]
dictx = {
    'opcodes': nOPCODES,
    'input': _c,
    'trace': nTRACES,
    'oplist': _s
}
scipy.io.savemat('m-no.mat',mdict=dictx)
scipy.io.savemat('state-no.mat',mdict={'state_input': nSTATEINP})

/tmp/ipykernel_36705/1195855378.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _c = np.zeros((1,3), dtype=np.object)
/tmp/ipykernel_36705/1195855378.py:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _s = np.zeros((1,5), dtype=np.object)


In [6]:
import numpy as np

N = 10
nIC = 5*5*5

N_TEST= N
N_INST= nIC

OPCODES_S = ['eors','lsls','str','ldr','movs'] 

N_R = N_INST*N_TEST#(N_INST+4)*N_TEST

def make_seeds(inp, seeds):
    seeds[0] = (inp & 0xff000000) >> 24
    seeds[1] = (inp & 0xff0000) >> 16
    seeds[2] = (inp & 0xff00) >> 8
    seeds[3] = (inp & 0xff)
def getmsbyte(i32):
    return (i32 >> 24) & 0xff
def movetomsbyte(i32):
    return i32 << 24
def make_int(inpbytes, indx):
    return (getmsbyte(inpbytes[indx]) << 24) | (getmsbyte(inpbytes[indx+1]) << 16) | (getmsbyte(inpbytes[indx+2]) << 8) | getmsbyte(inpbytes[indx+3])

nOPCODES = np.zeros((N_R,3))
nINPUT1 = np.zeros((N_R,2), dtype=np.double)
nINPUT2 = np.zeros((N_R,2), dtype=np.double)
nINPUT3 = np.zeros((N_R,2), dtype=np.double)
nTRACES = np.zeros((N_R,1), dtype=np.double)
nSTATEINP = np.zeros((N_R,2), dtype=np.double)

seeds = np.zeros((8,4), dtype=np.uint32)

g=[]
#print(np.shape(INPUTS))
for i in range(0, N_TEST):
    inp1 = INPUTS[i][0]
    inp2 = INPUTS[i][1]
    inp3 = INPUTS[i][2]
    inp4 = INPUTS[i][3]
    
    testinpints = inputints[i]
    prngout = movetomsbyte(testinpints)
    #print(len(testinpints))
    POI_IDX = 1
    for j in range(0, N_INST):
        
        # in ldr and str leakage happens in the later cycle rather
        # than in the current cycle. Therefore, below code overrides 
        # the power value from the subsequent instruction
        if OPCODES_S[OPCODES[j][1]-1]== "str" or  OPCODES_S[OPCODES[j][1]-1]== "ldr":
            POI_IDX=2
            
        nOPCODES[i*N_INST + j,0] = OPCODES[j][0] 
        nOPCODES[i*N_INST + j,1] = OPCODES[j][1]
        nOPCODES[i*N_INST + j,2] = OPCODES[j][2]
        
        nINPUT1[i*N_INST + j] = (make_int(prngout, 0), make_int(prngout, 4))#inp1
        nINPUT2[i*N_INST + j] = (make_int(prngout, 8), make_int(prngout, 12))#inp2
        nINPUT3[i*N_INST + j] = (make_int(prngout, 16), make_int(prngout, 20))#inp3
        
        nSTATEINP[i*N_INST + j] = (make_int(prngout, 24), make_int(prngout, 28))#inp4

        nTRACES[i*N_INST + j] = COMP[i][j][POI_IDX] / 100000.0
        #inp1 = tuple(map(incr,inp1))
        #inp2 = tuple(map(incr,inp2))
        #inp3 = tuple(map(incr,inp3))
        #inp4 = tuple(map(incr,inp4))
        #print(testinpints)
        
        testinpints = prng(testinpints)
        prngout = testinpints
    
    #for j in range(N_INST, N_INST+4):
        
    #    nTRACES[i*N_INST + j] = COMP[i][j][1] / 100000.0
        
    #    if _hd(inp1[0],inp2[1]) == 11:
    #        #print(_hd(inp1[0],inp2[1]))
    #        g.append(nTRACES[i*N_INST + j][0])
    #    #print(_hd(inp1[0],inp2[1]))
    #    inp1 = tuple(map(incr,inp1))
    #    inp2 = tuple(map(incr,inp2))
    #    inp3 = tuple(map(incr,inp3))
import scipy.io

_c = np.zeros((1,3), dtype=np.object)
_c[0][0] = nINPUT1
_c[0][1] = nINPUT2
_c[0][2] = nINPUT3
_s = np.zeros((1,5), dtype=np.object)
_s[0][0] = OPCODES_S[0]
_s[0][1] = OPCODES_S[1]
_s[0][2] = OPCODES_S[2]
_s[0][3] = OPCODES_S[3]
_s[0][4] = OPCODES_S[4]
dictx = {
    'opcodes': nOPCODES,
    'input': _c,
    'trace': nTRACES,
    'oplist': _s
}
scipy.io.savemat('m-%s.mat'%(state_dir.replace('-state','')),mdict=dictx)
scipy.io.savemat('state-%s.mat'%(state_dir.replace('-state','')),mdict={'state_input': nSTATEINP})



NameError: name 'INPUTS' is not defined